In [1]:
import matplotlib.pyplot as plt
import numpy as np
import earthpy.plot as ep
import seaborn as sns
import earthpy.spatial as es
import keras
import pandas as pd
from keras.layers import  Conv3D, Flatten, Dense, Reshape,MaxPool3D
from keras.layers import Dropout, Input, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from operator import truediv
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import init_notebook_mode
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral
from time import time
init_notebook_mode(connected=True)

In [2]:
## GLOBAL VARIABLES
dataset = 'SA'
test_ratio = 0.7
windowSize = 25

In [3]:
def loadData(name):
    if name=='SA':
        data = sio.loadmat('Salinas_corrected.mat')['salinas_corrected']
        labels = sio.loadmat('Salinas_gt.mat')['salinas_gt']
    
    
    return data,labels
data,labels=loadData(dataset)

In [4]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [5]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca


In [6]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [7]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX =padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [8]:
X,y = loadData(dataset)
X.shape, y.shape



((512, 217, 204), (512, 217))

In [9]:
K = X.shape[2]



In [10]:
K = 30 if dataset == 'IP' else 15
X,pca = applyPCA(X,numComponents=K)

In [11]:
X.shape

X, y = createImageCubes(X, y, windowSize=windowSize)


In [12]:
X.shape, y.shape

((54129, 25, 25, 15), (54129,))

In [13]:
X_train, X_test, y_train, y_test = splitTrainTestSet(X, y, test_ratio)
X_train, X_val, y_train, y_val = splitTrainTestSet(X_train, y_train, test_ratio)


In [14]:
X_train = X_train.reshape(-1, windowSize, windowSize, K, 1)
y_train.shape

(4871,)

In [15]:
y_train = to_categorical(y_train)
y_train.shape



(4871, 16)

In [16]:
y_val=to_categorical(y_val)
y_val.shape

(11367, 16)

In [17]:
y_test=to_categorical(y_test)
y_test.shape

(37891, 16)

In [18]:
y_train.shape, y_val.shape, y_test.shape

((4871, 16), (11367, 16), (37891, 16))

In [19]:
X_train.shape, X_val.shape, X_test.shape

((4871, 25, 25, 15, 1), (11367, 25, 25, 15), (37891, 25, 25, 15))

In [ ]:
S = windowSize
L = K
output_units =  9 if(dataset== 'PU' or dataset=='PC') else 16


In [ ]:
input_layer = Input((S, S, L, 1))

In [ ]:
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 7), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 5), activation='relu')(conv_layer1)
pooling_layer1 = MaxPool3D(pool_size=(2,2,2))(conv_layer2)


conv_layer3 = Conv3D(filters=32, kernel_size=(3,3,3), activation='relu')(conv_layer2)
pooling_layer2=MaxPool3D(pool_size=(2,2,2))(conv_layer3)
                     
pooling_layer2=BatchNormalization()(pooling_layer2)
                     

flatten_layer = Flatten()(pooling_layer2)

## fully connected layers
dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=output_units, activation='softmax')(dense_layer2)

model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
model.summary()

In [ ]:
adam = Adam(learning_rate=0.001, weight_decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor = 'val_loss',
                            mode = 'min',
                            min_delta = 0,
                            patience = 10,
                            restore_best_weights = True)

checkpoint = ModelCheckpoint(filepath = 'best-models1.keras', 
                             monitor = 'val_loss', 
                             mode ='min', 
                             save_best_only = True)

tensorboard = TensorBoard(log_dir='SA_logss/{}'.format(time()))

In [ ]:
filepath = "best-models1.keras"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
validation_data=(X_val,y_val)

In [ ]:
history = model.fit(x=X_train,y= y_train,epochs=120, batch_size=256,validation_data=(X_val,y_val), callbacks=callbacks_list)

In [ ]:
import pandas as pd
hist = pd.DataFrame(data= history.history)

fig = go.Figure()

fig.add_trace(go.Scatter(x = hist.index, y = hist.loss.values,
                    mode='lines+markers',
                    name='Train Loss'))

fig.add_trace(go.Scatter(x = hist.index, y = hist.accuracy.values,
                    mode='lines+markers',
                    name='Train Accuracy'))


fig.add_trace(go.Scatter(x = hist.index, y = hist.val_loss.values,
                    mode='lines+markers', name='Test loss'))

fig.add_trace(go.Scatter(x = hist.index, y = hist.val_accuracy.values,
                    mode='lines+markers', name='Test Accuracy'))


fig.show()

In [21]:
#model.load_weights("best-model1.keras")
from tensorflow.keras.models import load_model
model=load_model("best-models1.keras")
model.load_weights("best-models1.keras")


In [22]:
X_test = X_test.reshape(-1, windowSize, windowSize, K, 1)
X_test.shape


(37891, 25, 25, 15, 1)

In [23]:




y_test.shape

(37891, 16)

In [24]:
Y_pred_test = model.predict(X_test)
y_pred_test = np.argmax(Y_pred_test, axis=1)

1185/1185 [==============================] - 64s 53ms/step


In [25]:
classification = classification_report(np.argmax(y_test, axis=1), y_pred_test)
print(classification)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1406
           1       1.00      1.00      1.00      2608
           2       1.00      1.00      1.00      1383
           3       1.00      0.99      1.00       976
           4       1.00      1.00      1.00      1875
           5       1.00      1.00      1.00      2771
           6       1.00      1.00      1.00      2505
           7       1.00      0.99      0.99      7890
           8       1.00      1.00      1.00      4342
           9       1.00      1.00      1.00      2295
          10       1.00      0.99      1.00       748
          11       1.00      1.00      1.00      1349
          12       1.00      0.96      0.98       641
          13       0.96      1.00      0.98       749
          14       0.98      1.00      0.99      5088
          15       1.00      1.00      1.00      1265

    accuracy                           1.00     37891
   macro avg       1.00   

In [26]:
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [27]:
def reports (X_test,y_test,name):
    #start = time.time()
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)
    #end = time.time()
    #print(end - start)
    if name == 'SA':
        target_names = ['Brocoli_green_weeds_1','Brocoli_green_weeds_2','Fallow','Fallow_rough_plow','Fallow_smooth',
                        'Stubble','Celery','Grapes_untrained','Soil_vinyard_develop','Corn_senesced_green_weeds',
                        'Lettuce_romaine_4wk','Lettuce_romaine_5wk','Lettuce_romaine_6wk','Lettuce_romaine_7wk',
                        'Vinyard_untrained','Vinyard_vertical_trellis']
  
   
    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(y_test, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=64)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100
    
    return classification, confusion, Test_Loss, Test_accuracy, oa*100, each_acc*100, aa*100, kappa*100

In [ ]:
"""def reportsn(Xtest,ytest):
    
    
    Y_pred=model.predict(Xtest)
    y_pred = np.argmax (Y_pred, axis=1)
    
    target_names = ['Brocoli_green_weeds_1','Brocoli_green_weeds_2','Fallow','Fallow_rough_plow','Fallow_smooth',
                        'Stubble','Celery','Grapes_untrained','Soil_vinyard_develop','Corn_senesced_green_weeds',
                        'Lettuce_romaine_4wk','Lettuce_romaine_5wk','Lettuce_romaine_6wk','Lettuce_romaine_7wk',
                        'Vinyard_untrained','Vinyard_vertical_trellis']
  

    classification1 = classification_report(np.argmax (ytest, axis=1), y_pred, target_names=target_names) 
    confusion1=confusion_matrix (np.argmax (ytest, axis=1), y_pred)
    score1 = model.evaluate (Xtest, ytest, batch_size=64)
    Test_Loss1 =score1[0]*100
    Test_accuracy1 = score1[1]*100
    return classification1, confusion1, Test_Loss1, Test_accuracy1, score1"""

In [ ]:
classifcation1,confusion1, Test_Loss1, Test_accuracy1, score1=reportsn(Xtest,ytest)

print(classification,confusion1, Test_Loss1, Test_accuracy1, score1)




In [28]:
classification, confusion, Test_loss, Test_accuracy, oa, each_acc, aa, kappa = reports(X_test,y_test,dataset)
classification = str(classification)
confusion = str(confusion)
file_name = "classification_reportsal.txt"


593/593 [==============================] - 62s 103ms/step - loss: 0.0122 - accuracy: 0.9963


In [ ]:
with open(file_name, 'w') as x_file:
    x_file.write('{} Test loss (%)'.format(Test_loss))
    x_file.write('\n')
    x_file.write('{} Test accuracy (%)'.format(Test_accuracy))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{} Kappa accuracy (%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy (%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy (%)'.format(aa))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}'.format(classification))
    x_file.write('\n')
    x_file.write('{}'.format(confusion))
